In [1]:
from harvai.data import get_clean_preproc_data

In [2]:
data = get_clean_preproc_data()

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Text2TextGenerationPipeline

model_name = 'lincoln/barthez-squadFR-fquad-piaf-question-generation'

loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_name)




In [32]:
questions = []
articles = []
for article in range(0,1148):
    
    if(len(data.iloc[article,2])) > 1020: 
        text = data.iloc[article,2][0:1020] 
    else:    
        text = data.iloc[article,2]
        
    inputs = loaded_tokenizer(text, return_tensors='pt')

    out = loaded_model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_beams=5,
        num_return_sequences=5,
        length_penalty=20
    )


    for question in out:
        questions.append(loaded_tokenizer.decode(question, skip_special_tokens=True))
        articles.append(article)


In [4]:
import pandas as pd

In [33]:
df_questions =pd.DataFrame(columns=['questions','id'])
df_questions['questions'] =questions 
df_questions['id'] =articles 

In [39]:
df_questions.to_csv('../raw_data/generated_questions_dataset.csv')

In [48]:
test = pd.read_csv('../raw_data/generated_questions_dataset.csv')

In [51]:
test.drop(columns='Unnamed: 0')

,questions,id
0,"Quelle est la signification du terme ""véhicule à moteur""?",0
1,"Que signifie le terme ""véhicule à moteur"" en français?",0
2,"Quelle est la signification du terme ""véhicule à moteur""?",0
3,Quel est le terme utilisé pour désigner les véhicules à moteur?,0
4,"Que signifie le terme ""véhicule à moteur""?",0
...,...,...
5735,Quelles sont les mentions facultatives d'un modèle de fiche descriptive de l...,1147
5736,Qu'est-ce qui doit être mentionné dans la fiche descriptive de l état du véh...,1147
5737,Qu'est-ce qui doit être mentionné sur la fiche descriptive de l état du véhi...,1147
5738,Quelles sont les mentions facultatives d'un modèle de fiche descriptive de l...,1147


In [38]:
df_questions

,questions,id
0,"Quelle est la signification du terme ""véhicule à moteur""?",0
1,"Que signifie le terme ""véhicule à moteur"" en français?",0
2,"Quelle est la signification du terme ""véhicule à moteur""?",0
3,Quel est le terme utilisé pour désigner les véhicules à moteur?,0
4,"Que signifie le terme ""véhicule à moteur""?",0
...,...,...
5735,Quelles sont les mentions facultatives d'un modèle de fiche descriptive de l...,1147
5736,Qu'est-ce qui doit être mentionné dans la fiche descriptive de l état du véh...,1147
5737,Qu'est-ce qui doit être mentionné sur la fiche descriptive de l état du véhi...,1147
5738,Quelles sont les mentions facultatives d'un modèle de fiche descriptive de l...,1147


In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from harvai.bm25 import Bm25
from harvai.nn_model import Nn_model
from harvai.embedding import Embedding
from harvai.dpr import DPR
from harvai.utils import score

In [9]:
nn = Nn_model(5)
nn.clean_data()
nn.fit()

In [13]:
embedding = Embedding(5)
embedding.clean_data()
embedding.fit()

In [10]:
model = Bm25(5)
model.clean_data()
model.fit()

In [16]:
dpr = DPR(5)
dpr.clean_data()
dpr.fit()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../raw_data/retriever.pt/query_encoder
WARNING - haystack.modeling.model.language_model -  Using a model of type 'camembert' which might be incompatible with DPR encoders.Bert based encoders are supported that need input_ids,token_type_ids,attention_mask as input tensors.
INFO - haystack.modeling.model.language_model -  Loaded ../raw_data/retriever.pt/query_encoder
The tokenizer class

In [7]:
score(nn)

2871it [00:05, 487.88it/s]                          


0.6335771508185302

In [11]:
score(model)

2871it [00:11, 246.69it/s]                          


0.6565656565656566

In [17]:
score(dpr)

2871it [11:44,  4.08it/s]                          


0.6078021595262975

In [18]:
score(embedding)

2871it [06:53,  6.94it/s]                          


0.5806339254615117